<font size="5">**Project Summary**</font><br><br>
In this competition, the main task is to do surface classification given robot sensors' time series data. 

After reviewing the notebook, the champion of this competition is to use fourier analysis [https://www.kaggle.com/trohwer64/simple-fourier-analysis](http://). Other solutions evolve classical machine learning algorithms, e.g Random Forest, Decision Trees... They have achieved good performance. 

Traditionally time series classification includes Fourier Transform or feature engineering. Although they can perform well in small datasets, they are heuristic and not task dependent. Traditional machine learning skills may not have the same representation power compared to deep learning. In this project, I mainly explore ways to solve the problem by using deep learning. After searching papers at [https://scholar.google.com/](http://) I found that 1d convolution neural network**[1][2]** is a good technique when doing the time series classification job. So I apply 1d convolutional neural network to solve this problem. The core code is based on pytorch. 



Reference Papers:<br>
*[1] Yi Zheng, Qi Liu, Enhong Chen, Yong Ge, J. Leon Zhao, Time Series Classification Using Multi-Channels Deep Convolutional Neural Networks, IJCAI 2015<br>
[2] Jian Bo Yang, Minh Nhut Nguyen, Phyo Phyo San, Xiao Li Li, Shonali Krishnaswamy, Deep Convolutional Neural Networks On Multichannel Time Series
For Human Activity Recognition, IJCAI 2015<br>*

<font size="5">**External libs I use**</font><br>

In [195]:
# libs we are using
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader

<font size="5">**Dataset and Data Preprocessing**</font><br>

For dataset, the original data includes 7816 different time series(First 3810 samples are training data with labels, following 3816 are testing data without labels). Since the professor said there may have some bugs in Kaggle's grading system. We only use original training data right now.

Compared with other methods which firstly prepross data using fourier transformation. I just unify data between 0 and 1 and feed this simply preprocessed data into the neural network. The superising result is that those two methods have similar performance when doing cross validation if feeding different proprecessed data into the same network architecture. This phenomenon seems to tell us that if the neural network is strong enough, it can automatically catch important features in the data.

In this project, I use the raw data as input of the network. I concatenated all same time series data into one single numpy array. In total there are 7816 time series. The first 3810 rows are training data with given labels, the rest 3816 rows are testing data where labels are not given. In total raw data has 7816 rows. In addition to that, we split the raw training data into training data and validation data by the ratio of 80/20.


I create three datasets and data loaders for them to make the data ready for model's training. The process is straightforward. I split the labelled data into two subsets, and keep testing data as is. Also, I convert Numpy arrays into `torch.tensor` objects of proper type (float for samples, and long - for targets).


In [196]:
def create_datasets(data, target, train_size, valid_pct=0.2, seed=None):
    """Converts NumPy arrays into PyTorch datsets.
    
    Three datasets are created in total:
        * training dataset
        * validation dataset
        * testing (un-labelled) dataset

    """
    raw =  data
    sz = train_size
    idx = np.arange(sz)
    trn_idx, val_idx = train_test_split(
        idx, test_size=valid_pct, random_state=seed)
     
    trn_ds = TensorDataset(
        torch.tensor(raw[:sz][trn_idx]).float(), 
        torch.tensor(target[:sz][trn_idx]).long())
    val_ds = TensorDataset(
        torch.tensor(raw[:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    tst_ds = TensorDataset(
        torch.tensor(raw[sz:]).float(), 
        torch.tensor(target[sz:]).long())
    return trn_ds, val_ds, tst_ds

# We use this function to divide the data
def create_datasets2(data_arr, train_size, valid_pct=0.2, seed=None):
    """Converts NumPy arrays into PyTorch datsets.
    
    Three datasets are created in total:
        * training dataset
        * validation dataset
        * testing (un-labelled) dataset

    """
    raw =  data_arr
    sz = train_size

    idx = np.arange(sz)
    print("total data length")
    print(len(idx))
    trn_idx, tst_idx = train_test_split(idx, test_size=0.2, random_state=1)
    trn_idx, val_idx = train_test_split(trn_idx, test_size=0.25, random_state=1)
    
    print("input one dimension shape")
    print(raw[0].shape)
    print("training data length")
    print(len(trn_idx))
    print("validation data le")
    print(len(val_idx))
    print("testing d")
    print(len(tst_idx))
    
    trn_ds = TensorDataset(
        torch.tensor(raw[0][:sz][trn_idx]).float(),
        torch.tensor(raw[1][:sz][trn_idx]).float(),
        torch.tensor(raw[2][:sz][trn_idx]).float(),
        torch.tensor(raw[3][:sz][trn_idx]).float(),
        torch.tensor(raw[4][:sz][trn_idx]).float(),
        torch.tensor(raw[5][:sz][trn_idx]).float(),
        torch.tensor(raw[6][:sz][trn_idx]).float(),
        torch.tensor(raw[7][:sz][trn_idx]).float(),
        torch.tensor(raw[8][:sz][trn_idx]).float(),
        torch.tensor(target[:sz][trn_idx]).long())
    
    val_ds = TensorDataset(
        torch.tensor(raw[0][:sz][val_idx]).float(),
        torch.tensor(raw[1][:sz][val_idx]).float(), 
        torch.tensor(raw[2][:sz][val_idx]).float(), 
        torch.tensor(raw[3][:sz][val_idx]).float(), 
        torch.tensor(raw[4][:sz][val_idx]).float(), 
        torch.tensor(raw[5][:sz][val_idx]).float(), 
        torch.tensor(raw[6][:sz][val_idx]).float(), 
        torch.tensor(raw[7][:sz][val_idx]).float(), 
        torch.tensor(raw[8][:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    
    tst_ds = TensorDataset(
        torch.tensor(raw[0][:sz][tst_idx]).float(),
        torch.tensor(raw[1][:sz][tst_idx]).float(),
        torch.tensor(raw[2][:sz][tst_idx]).float(),
        torch.tensor(raw[3][:sz][tst_idx]).float(),
        torch.tensor(raw[4][:sz][tst_idx]).float(),
        torch.tensor(raw[5][:sz][tst_idx]).float(),
        torch.tensor(raw[6][:sz][tst_idx]).float(),
        torch.tensor(raw[7][:sz][tst_idx]).float(),
        torch.tensor(raw[8][:sz][tst_idx]).float(),
        torch.tensor(target[:sz][tst_idx]).long())
    
    return trn_ds, val_ds, tst_ds

def create_loaders(data, bs=128, jobs=0):
    """Wraps the datasets returned by create_datasets function with data loaders."""
    
    trn_ds, val_ds, tst_ds = data
    trn_dl = DataLoader(trn_ds, batch_size=bs, shuffle=True, num_workers=jobs)
    val_dl = DataLoader(val_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    tst_dl = DataLoader(tst_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    return trn_dl, val_dl, tst_dl

<font size="5">**Neural Network Architecture**</font><br>

In [197]:
class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [198]:
class Surface_Classifier(nn.Module):
    def __init__(self, raw_ni, no, drop=.5):
        super().__init__()
        
        self.conv1d_channel_0 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
           
        
        )
            
            
        
        self.conv1d_channel_1 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            
        )
        
        self.conv1d_channel_2 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
           
        )
        
        self.conv1d_channel_3 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
           
        )
        
        self.conv1d_channel_4 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
        )
            
            
        
        self.conv1d_channel_5 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
        )
        
        self.conv1d_channel_6 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
        )
        
        self.conv1d_channel_7 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
            
        )
            

       
        
        self.conv1d_channel_8 = nn.Sequential(
            nn.Conv1d(1, 8, 16, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),

            nn.Conv1d(8, 16, 11, 3, 3),
            nn.ReLU(),
            nn.Dropout(),
            
            nn.MaxPool1d(kernel_size=2),
            
           
        )

            

        self.dense = nn.Sequential(
            nn.Linear(432, 144),  nn.ReLU(),nn.Dropout(),
            nn.Linear(144, 64),  nn.ReLU(),nn.Dropout(),
            nn.Linear(64, no),  nn.ReLU()
        )
        
            

    def forward(self, t_channel_0, t_channel_1, t_channel_2, t_channel_3, t_channel_4, t_channel_5, t_channel_6, t_channel_7, t_channel_8):
        conv1d_out_channel_0 = self.conv1d_channel_0(t_channel_0)
        conv1d_out_channel_1 = self.conv1d_channel_1(t_channel_1)
        conv1d_out_channel_2 = self.conv1d_channel_2(t_channel_2)
        conv1d_out_channel_3 = self.conv1d_channel_3(t_channel_3)
        conv1d_out_channel_4 = self.conv1d_channel_4(t_channel_4)
        conv1d_out_channel_5 = self.conv1d_channel_5(t_channel_5)
        conv1d_out_channel_6 = self.conv1d_channel_6(t_channel_6)
        conv1d_out_channel_7 = self.conv1d_channel_7(t_channel_7)
        conv1d_out_channel_8 = self.conv1d_channel_8(t_channel_8)
        
        t_in = torch.cat([conv1d_out_channel_0,conv1d_out_channel_1, conv1d_out_channel_2, conv1d_out_channel_3, conv1d_out_channel_4, conv1d_out_channel_5, conv1d_out_channel_6, conv1d_out_channel_7, conv1d_out_channel_8], dim=1)
        res = t_in.view(t_in.size(0), -1)
        out = self.dense(res)
        return out
        

<font size="5">**Initial setup for random seed and project path**</font><br>

In [199]:

# set up the seed
seed = 1
np.random.seed(seed)

# path to sample submission
sample = Path.cwd().parent/'input'/'career-con-2019'/'sample_submission.csv'

ROOT = Path.cwd().parent/'input'/'career-con-2019-preprocessed-data'
raw_arr = np.load(ROOT/'feat.npy').transpose(0, 2, 1)
target = np.load(ROOT/'target.npy')

raw_dim_data = [None]*9
#print(raw_dim_data)
for i in range(0, 9):
    raw_dim_data[i] = raw_arr[:,i,:]
#    print("raw data shape")
    
    raw_dim_data[i] = raw_dim_data[i].reshape([7626,1,128])
#    print(raw_dim_data[i].shape)
    
# print("raw array shape")
# print(raw_arr.shape)
# print("label array shape")
# print(target.shape)

trn_sz = 3810  # only the first `trn_sz` rows in each array include labelled data
#datasets = create_datasets((raw_arr), target, trn_sz, seed=seed)
datasets = create_datasets2((raw_dim_data), trn_sz, seed=seed)


# make sure that we run on a proper device (not relevant for Kaggle kernels but helpful in Jupyter sessions)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


trn_dl, val_dl, tst_dl = create_loaders(datasets, bs=256)

total data length
3810
input one dimension shape
(7626, 1, 128)
training data length
2286
validation data le
762
testing d
762


<font size="5">**Training model**</font><br>
Now everything is ready to create a training loop and see if our model works. For each training epoch, the loop performs the following actions:
1. train model on the `trn_ds` dataset;
2. verify quality on the `val_ds` dataset;
3. check if the quality improved since previous epoch, and if so, save the model's weights onto disk;
4. in case if the model's quality isn't impoving for `patience` epochs, the training is stopped.
Also, the code tracks loss and accuracy history, and prints current scores with exponentially increasing logging frequency, i.e., only at 1, 2, 4, 8... epochs.

In [200]:
raw_feat = raw_arr.shape[1]


lr = 0.002
n_epochs = 10000
iterations_per_epoch = len(trn_dl)
num_classes = 9
best_acc = 0
patience, trials = 500, 0
base = 1
step = 2
loss_history = []
acc_history = []

model = Surface_Classifier(raw_feat, num_classes).to(device)
model.cuda()
criterion = nn.CrossEntropyLoss(reduction='sum')
opt = optim.Adam(model.parameters(), lr=lr)

print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    model.train()
    epoch_loss = 0
    for i, batch in enumerate(trn_dl):
        #x_raw, y_batch = [t.to(device) for t in batch]
        x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch = [t.to(device) for t in batch]
        opt.zero_grad()
        
#         print("channel 0 type")
#         print(type(x_channel_0))
#         print("channel 0 shape")
#         print(x_channel_0.shape)
#         print("batch type")
#         print(type(batch))
#         print(len(batch))
#         print(batch[0].shape)
#         print(batch[9].shape)

        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        
        
#         out = []
#         with torch.no_grad():
#             for x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch in batch:
#                 output = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
#                 out.append(output.detach())
#         out = torch.cat(out)
        

    
        loss = criterion(out, y_batch)
        epoch_loss += loss.item()
        loss.backward()
        opt.step()
        
    epoch_loss /= trn_sz
    loss_history.append(epoch_loss)
    
    model.eval()
    correct, total = 0, 0
    
    for batch in val_dl:
        x_channel_0, x_channel_1,  x_channel_2,  x_channel_3,  x_channel_4,  x_channel_5,  x_channel_6,  x_channel_7,  x_channel_8, y_batch = [t.to(device) for t in batch]
        
        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
    acc = correct / total
    acc_history.append(acc)

    if epoch % base == 0:
        print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')
        base *= step
    print(f'Epoch: {epoch:3d}. Loss: {epoch_loss:.4f}. Acc.: {acc:2.2%}')

    
    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break
            
print('Done!')

Start model training
Epoch:   1. Loss: 1.2813. Acc.: 21.78%
Epoch:   1. Loss: 1.2813. Acc.: 21.78%
Epoch 1 best model saved with accuracy: 21.78%
Epoch:   2. Loss: 1.2446. Acc.: 30.45%
Epoch:   2. Loss: 1.2446. Acc.: 30.45%
Epoch 2 best model saved with accuracy: 30.45%
Epoch:   3. Loss: 1.2006. Acc.: 35.83%
Epoch 3 best model saved with accuracy: 35.83%
Epoch:   4. Loss: 1.1605. Acc.: 34.65%
Epoch:   4. Loss: 1.1605. Acc.: 34.65%
Epoch:   5. Loss: 1.1238. Acc.: 36.22%
Epoch 5 best model saved with accuracy: 36.22%
Epoch:   6. Loss: 1.0983. Acc.: 38.19%
Epoch 6 best model saved with accuracy: 38.19%
Epoch:   7. Loss: 1.0753. Acc.: 43.96%
Epoch 7 best model saved with accuracy: 43.96%
Epoch:   8. Loss: 1.0632. Acc.: 44.36%
Epoch:   8. Loss: 1.0632. Acc.: 44.36%
Epoch 8 best model saved with accuracy: 44.36%
Epoch:   9. Loss: 1.0452. Acc.: 45.80%
Epoch 9 best model saved with accuracy: 45.80%
Epoch:  10. Loss: 1.0404. Acc.: 46.33%
Epoch 10 best model saved with accuracy: 46.33%
Epoch:  1

Epoch: 149. Loss: 0.5071. Acc.: 70.08%
Epoch: 150. Loss: 0.5137. Acc.: 68.50%
Epoch: 151. Loss: 0.5036. Acc.: 69.03%
Epoch: 152. Loss: 0.5148. Acc.: 70.47%
Epoch 152 best model saved with accuracy: 70.47%
Epoch: 153. Loss: 0.4882. Acc.: 70.08%
Epoch: 154. Loss: 0.5080. Acc.: 69.82%
Epoch: 155. Loss: 0.4953. Acc.: 70.60%
Epoch 155 best model saved with accuracy: 70.60%
Epoch: 156. Loss: 0.5121. Acc.: 69.03%
Epoch: 157. Loss: 0.5205. Acc.: 70.08%
Epoch: 158. Loss: 0.5031. Acc.: 72.05%
Epoch 158 best model saved with accuracy: 72.05%
Epoch: 159. Loss: 0.4924. Acc.: 69.16%
Epoch: 160. Loss: 0.4947. Acc.: 69.69%
Epoch: 161. Loss: 0.4998. Acc.: 70.08%
Epoch: 162. Loss: 0.4916. Acc.: 71.78%
Epoch: 163. Loss: 0.4985. Acc.: 69.82%
Epoch: 164. Loss: 0.4885. Acc.: 70.73%
Epoch: 165. Loss: 0.5018. Acc.: 70.08%
Epoch: 166. Loss: 0.4889. Acc.: 70.73%
Epoch: 167. Loss: 0.4789. Acc.: 69.55%
Epoch: 168. Loss: 0.4924. Acc.: 71.65%
Epoch: 169. Loss: 0.4868. Acc.: 69.55%
Epoch: 170. Loss: 0.4807. Acc.: 71

Epoch: 345. Loss: 0.3873. Acc.: 75.59%
Epoch: 346. Loss: 0.3843. Acc.: 77.17%
Epoch: 347. Loss: 0.3605. Acc.: 75.72%
Epoch: 348. Loss: 0.3776. Acc.: 75.72%
Epoch: 349. Loss: 0.3683. Acc.: 75.85%
Epoch: 350. Loss: 0.3796. Acc.: 76.38%
Epoch: 351. Loss: 0.3732. Acc.: 75.85%
Epoch: 352. Loss: 0.3624. Acc.: 76.51%
Epoch: 353. Loss: 0.3599. Acc.: 76.51%
Epoch: 354. Loss: 0.3569. Acc.: 76.25%
Epoch: 355. Loss: 0.3546. Acc.: 75.59%
Epoch: 356. Loss: 0.3674. Acc.: 75.33%
Epoch: 357. Loss: 0.3694. Acc.: 74.93%
Epoch: 358. Loss: 0.3562. Acc.: 75.07%
Epoch: 359. Loss: 0.3644. Acc.: 75.46%
Epoch: 360. Loss: 0.3648. Acc.: 76.51%
Epoch: 361. Loss: 0.3534. Acc.: 76.77%
Epoch: 362. Loss: 0.3483. Acc.: 76.90%
Epoch: 363. Loss: 0.3480. Acc.: 74.80%
Epoch: 364. Loss: 0.3569. Acc.: 76.51%
Epoch: 365. Loss: 0.3909. Acc.: 75.85%
Epoch: 366. Loss: 0.3668. Acc.: 76.64%
Epoch: 367. Loss: 0.3561. Acc.: 75.33%
Epoch: 368. Loss: 0.3698. Acc.: 75.98%
Epoch: 369. Loss: 0.3438. Acc.: 76.77%
Epoch: 370. Loss: 0.3382.

Epoch: 552. Loss: 0.3317. Acc.: 78.48%
Epoch: 553. Loss: 0.3120. Acc.: 76.90%
Epoch: 554. Loss: 0.3168. Acc.: 78.61%
Epoch: 555. Loss: 0.3155. Acc.: 77.69%
Epoch: 556. Loss: 0.3289. Acc.: 77.56%
Epoch: 557. Loss: 0.2954. Acc.: 77.03%
Epoch: 558. Loss: 0.3088. Acc.: 76.77%
Epoch: 559. Loss: 0.3219. Acc.: 77.30%
Epoch: 560. Loss: 0.3189. Acc.: 77.30%
Epoch: 561. Loss: 0.3220. Acc.: 77.56%
Epoch: 562. Loss: 0.3215. Acc.: 75.98%
Epoch: 563. Loss: 0.3054. Acc.: 77.30%
Epoch: 564. Loss: 0.3281. Acc.: 77.17%
Epoch: 565. Loss: 0.3186. Acc.: 76.51%
Epoch: 566. Loss: 0.3057. Acc.: 76.90%
Epoch: 567. Loss: 0.3081. Acc.: 76.77%
Epoch: 568. Loss: 0.3179. Acc.: 77.82%
Epoch: 569. Loss: 0.3417. Acc.: 77.82%
Epoch: 570. Loss: 0.3241. Acc.: 77.95%
Epoch: 571. Loss: 0.2979. Acc.: 78.74%
Epoch: 572. Loss: 0.3237. Acc.: 76.51%
Epoch: 573. Loss: 0.3163. Acc.: 76.64%
Epoch: 574. Loss: 0.2853. Acc.: 77.03%
Epoch: 575. Loss: 0.3126. Acc.: 77.69%
Epoch: 576. Loss: 0.2930. Acc.: 78.35%
Epoch: 577. Loss: 0.3110.

Epoch: 760. Loss: 0.3018. Acc.: 78.08%
Epoch: 761. Loss: 0.2917. Acc.: 77.69%
Epoch: 762. Loss: 0.2945. Acc.: 77.82%
Epoch: 763. Loss: 0.2670. Acc.: 78.22%
Epoch: 764. Loss: 0.2779. Acc.: 77.69%
Epoch: 765. Loss: 0.2815. Acc.: 77.17%
Epoch: 766. Loss: 0.2700. Acc.: 78.61%
Epoch: 767. Loss: 0.2868. Acc.: 78.35%
Epoch: 768. Loss: 0.2689. Acc.: 78.48%
Epoch: 769. Loss: 0.2716. Acc.: 77.30%
Epoch: 770. Loss: 0.2964. Acc.: 78.08%
Epoch: 771. Loss: 0.2896. Acc.: 78.08%
Epoch: 772. Loss: 0.2833. Acc.: 77.03%
Epoch: 773. Loss: 0.2645. Acc.: 76.77%
Epoch: 774. Loss: 0.2857. Acc.: 78.08%
Epoch: 775. Loss: 0.2894. Acc.: 77.30%
Epoch: 776. Loss: 0.2936. Acc.: 77.82%
Epoch: 777. Loss: 0.2701. Acc.: 77.95%
Epoch: 778. Loss: 0.2857. Acc.: 78.08%
Epoch: 779. Loss: 0.2761. Acc.: 77.30%
Epoch: 780. Loss: 0.3000. Acc.: 77.17%
Epoch: 781. Loss: 0.2863. Acc.: 77.82%
Epoch: 782. Loss: 0.2601. Acc.: 77.03%
Epoch: 783. Loss: 0.3116. Acc.: 77.95%
Epoch: 784. Loss: 0.2777. Acc.: 76.77%
Epoch: 785. Loss: 0.3050.

...

Epoch: 1167. Loss: 0.2460. Acc.: 78.61%
Epoch: 1168. Loss: 0.2336. Acc.: 79.53%
Epoch: 1169. Loss: 0.2545. Acc.: 79.92%
Epoch: 1170. Loss: 0.2558. Acc.: 79.27%
Epoch: 1171. Loss: 0.2699. Acc.: 78.35%
Epoch: 1172. Loss: 0.2521. Acc.: 80.05%
Epoch: 1173. Loss: 0.2510. Acc.: 78.48%
Epoch: 1174. Loss: 0.2627. Acc.: 78.87%
Epoch: 1175. Loss: 0.2566. Acc.: 78.22%
Epoch: 1176. Loss: 0.2605. Acc.: 78.48%
Epoch: 1177. Loss: 0.2709. Acc.: 79.00%
Epoch: 1178. Loss: 0.2711. Acc.: 79.66%
Epoch: 1179. Loss: 0.2355. Acc.: 79.40%
Epoch: 1180. Loss: 0.2473. Acc.: 79.92%
Epoch: 1181. Loss: 0.2555. Acc.: 79.27%
Epoch: 1182. Loss: 0.2706. Acc.: 77.82%
Epoch: 1183. Loss: 0.2495. Acc.: 79.53%
Epoch: 1184. Loss: 0.2654. Acc.: 80.71%
Epoch: 1185. Loss: 0.2619. Acc.: 80.58%
Epoch: 1186. Loss: 0.2521. Acc.: 79.40%
Epoch: 1187. Loss: 0.2356. Acc.: 79.92%
Epoch: 1188. Loss: 0.2346. Acc.: 79.40%
Epoch: 1189. Loss: 0.2475. Acc.: 79.79%
Epoch: 1190. Loss: 0.2208. Acc.: 78.87%
Epoch: 1191. Loss: 0.2510. Acc.: 79.40%


Epoch: 1370. Loss: 0.2424. Acc.: 78.61%
Epoch: 1371. Loss: 0.2519. Acc.: 80.18%
Epoch: 1372. Loss: 0.2591. Acc.: 79.40%
Epoch: 1373. Loss: 0.2537. Acc.: 77.56%
Epoch: 1374. Loss: 0.2362. Acc.: 76.77%
Epoch: 1375. Loss: 0.2232. Acc.: 79.40%
Epoch: 1376. Loss: 0.2491. Acc.: 78.48%
Epoch: 1377. Loss: 0.2396. Acc.: 77.82%
Epoch: 1378. Loss: 0.2356. Acc.: 79.00%
Epoch: 1379. Loss: 0.2427. Acc.: 79.66%
Epoch: 1380. Loss: 0.2508. Acc.: 78.22%
Epoch: 1381. Loss: 0.2378. Acc.: 78.87%
Epoch: 1382. Loss: 0.2460. Acc.: 78.22%
Epoch: 1383. Loss: 0.2513. Acc.: 79.40%
Epoch: 1384. Loss: 0.2340. Acc.: 78.74%
Epoch: 1385. Loss: 0.2315. Acc.: 79.66%
Epoch: 1386. Loss: 0.2435. Acc.: 78.61%
Epoch: 1387. Loss: 0.2501. Acc.: 78.87%
Epoch: 1388. Loss: 0.2424. Acc.: 79.79%
Epoch: 1389. Loss: 0.2459. Acc.: 80.05%
Epoch: 1390. Loss: 0.2565. Acc.: 78.35%
Epoch: 1391. Loss: 0.2352. Acc.: 78.35%
Epoch: 1392. Loss: 0.2505. Acc.: 79.40%
Epoch: 1393. Loss: 0.2206. Acc.: 79.27%
Epoch: 1394. Loss: 0.2299. Acc.: 79.40%


Epoch: 1572. Loss: 0.2376. Acc.: 80.71%
Epoch: 1573. Loss: 0.2272. Acc.: 81.10%
Epoch: 1574. Loss: 0.2288. Acc.: 81.89%
Epoch 1574 best model saved with accuracy: 81.89%
Epoch: 1575. Loss: 0.2249. Acc.: 80.31%
Epoch: 1576. Loss: 0.2619. Acc.: 81.50%
Epoch: 1577. Loss: 0.2294. Acc.: 81.63%
Epoch: 1578. Loss: 0.2214. Acc.: 81.50%
Epoch: 1579. Loss: 0.2248. Acc.: 82.68%
Epoch 1579 best model saved with accuracy: 82.68%
Epoch: 1580. Loss: 0.2221. Acc.: 81.89%
Epoch: 1581. Loss: 0.2411. Acc.: 80.84%
Epoch: 1582. Loss: 0.2490. Acc.: 80.18%
Epoch: 1583. Loss: 0.2512. Acc.: 80.84%
Epoch: 1584. Loss: 0.2490. Acc.: 79.92%
Epoch: 1585. Loss: 0.2415. Acc.: 81.23%
Epoch: 1586. Loss: 0.2407. Acc.: 80.84%
Epoch: 1587. Loss: 0.2347. Acc.: 80.05%
Epoch: 1588. Loss: 0.2375. Acc.: 80.84%
Epoch: 1589. Loss: 0.2306. Acc.: 81.50%
Epoch: 1590. Loss: 0.2425. Acc.: 81.10%
Epoch: 1591. Loss: 0.2567. Acc.: 80.45%
Epoch: 1592. Loss: 0.2423. Acc.: 80.97%
Epoch: 1593. Loss: 0.2284. Acc.: 81.50%
Epoch: 1594. Loss: 0

Epoch: 1775. Loss: 0.2376. Acc.: 81.89%
Epoch: 1776. Loss: 0.2268. Acc.: 82.68%
Epoch: 1777. Loss: 0.2262. Acc.: 80.45%
Epoch: 1778. Loss: 0.2211. Acc.: 80.71%
Epoch: 1779. Loss: 0.2160. Acc.: 79.92%
Epoch: 1780. Loss: 0.2319. Acc.: 80.71%
Epoch: 1781. Loss: 0.2306. Acc.: 79.66%
Epoch: 1782. Loss: 0.2424. Acc.: 80.45%
Epoch: 1783. Loss: 0.2233. Acc.: 80.84%
Epoch: 1784. Loss: 0.2233. Acc.: 80.97%
Epoch: 1785. Loss: 0.2446. Acc.: 81.23%
Epoch: 1786. Loss: 0.2441. Acc.: 80.18%
Epoch: 1787. Loss: 0.2194. Acc.: 79.13%
Epoch: 1788. Loss: 0.2410. Acc.: 80.84%
Epoch: 1789. Loss: 0.2295. Acc.: 80.84%
Epoch: 1790. Loss: 0.2309. Acc.: 80.71%
Epoch: 1791. Loss: 0.2397. Acc.: 80.05%
Epoch: 1792. Loss: 0.2062. Acc.: 79.92%
Epoch: 1793. Loss: 0.2340. Acc.: 80.05%
Epoch: 1794. Loss: 0.2231. Acc.: 80.45%
Epoch: 1795. Loss: 0.2197. Acc.: 79.79%
Epoch: 1796. Loss: 0.2245. Acc.: 79.92%
Epoch: 1797. Loss: 0.2218. Acc.: 79.00%
Epoch: 1798. Loss: 0.2484. Acc.: 79.27%
Epoch: 1799. Loss: 0.2328. Acc.: 79.66%


Epoch: 1980. Loss: 0.2403. Acc.: 79.40%
Epoch: 1981. Loss: 0.2184. Acc.: 79.66%
Epoch: 1982. Loss: 0.2438. Acc.: 80.45%
Epoch: 1983. Loss: 0.2203. Acc.: 79.92%
Epoch: 1984. Loss: 0.2024. Acc.: 80.05%
Epoch: 1985. Loss: 0.2148. Acc.: 79.40%
Epoch: 1986. Loss: 0.2143. Acc.: 80.18%
Epoch: 1987. Loss: 0.2280. Acc.: 79.66%
Epoch: 1988. Loss: 0.2232. Acc.: 79.92%
Epoch: 1989. Loss: 0.2322. Acc.: 80.31%
Epoch: 1990. Loss: 0.2168. Acc.: 80.58%
Epoch: 1991. Loss: 0.2025. Acc.: 80.84%
Epoch: 1992. Loss: 0.2559. Acc.: 80.45%
Epoch: 1993. Loss: 0.2216. Acc.: 80.45%
Epoch: 1994. Loss: 0.2387. Acc.: 80.05%
Epoch: 1995. Loss: 0.2288. Acc.: 81.36%
Epoch: 1996. Loss: 0.2159. Acc.: 79.53%
Epoch: 1997. Loss: 0.2069. Acc.: 80.18%
Epoch: 1998. Loss: 0.2217. Acc.: 80.18%
Epoch: 1999. Loss: 0.2219. Acc.: 79.66%
Epoch: 2000. Loss: 0.2320. Acc.: 78.74%
Epoch: 2001. Loss: 0.2177. Acc.: 80.71%
Epoch: 2002. Loss: 0.2372. Acc.: 80.84%
Epoch: 2003. Loss: 0.2371. Acc.: 79.40%
Epoch: 2004. Loss: 0.2252. Acc.: 80.05%


<font size="5">**Compute result on testing data**</font><br>

In [201]:
test_results = []
model.load_state_dict(torch.load('best.pth'))
model.eval()

# for batch in val_dl:
#     x_raw, y_batch = [t.to(device) for t in batch]
#     out = model(x_raw)
#     preds = F.log_softmax(out, dim=1).argmax(dim=1)
#     total += y_batch.size(0)
#     correct += (preds == y_batch).sum().item()
#     acc = correct / total
#     acc_history.append(acc)

# for batch in tst_dl:
#     x, y_batche = [t.to(device) for t in (batch)]
#     out = model(x)
#     y_hat = F.log_softmax(out, dim=1).argmax(dim=1)
#     test_results.extend(y_hat.tolist())
    
for batch in tst_dl:
        x_raw, y_batch = [t.to(device) for t in batch]
        out = model(x_channel_0, x_channel_1, x_channel_2, x_channel_3, x_channel_4, x_channel_5, x_channel_6, x_channel_7, x_channel_8)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
acc = correct / total
print("accurancy on test data: "+str(acc))

ValueError: too many values to unpack (expected 2)

<font size="5">**Store final result**</font><br>

In [ ]:
'''
submit = pd.read_csv(sample)
enc = joblib.load(ROOT/'encoder.model')
submit['surface'] = enc.inverse_transform(test_results)
submit.to_csv('submit_base.csv', index=None)
print("store result successfully!")
'''